In [3]:
import pandas as pd

In [4]:
bogota = pd.read_csv('Bogota_lat_lng.csv')

In [5]:
bogota.head()

,COUNTRY,CITY_NAME,LAT,LNG
0,CO,Bogotá,4.642811,-74.144544
1,CO,Bogotá,4.641695,-74.060087
2,CO,Bogotá,4.684074,-74.091357
3,CO,Bogotá,4.619381,-74.067988
4,CO,Bogotá,4.603926,-74.177953


In [6]:
import requests
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm
from getpass import getpass  

API_KEY = getpass("Introduce tu API Key de OpenWeather: ") 
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

def get_weather_data(lat, lon):
    """Fetch detailed weather data from OpenWeather API."""
    params = {
        "lat": lat,
        "lon": lon,
        "appid": API_KEY,
        "units": "metric",  
    }

    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return {
            "Latitude": lat,
            "Longitude": lon,
            "City": data.get("name", ""),
            "Country": data["sys"].get("country", ""),
            "Weather": data["weather"][0]["main"],
            "Description": data["weather"][0]["description"],
            "Temperature (°C)": data["main"]["temp"],
            "Feels Like (°C)": data["main"]["feels_like"],
            "Min Temp (°C)": data["main"]["temp_min"],
            "Max Temp (°C)": data["main"]["temp_max"],
            "Pressure (hPa)": data["main"]["pressure"],
            "Humidity (%)": data["main"]["humidity"],
            "Wind Speed (m/s)": data["wind"]["speed"],
            "Wind Direction (°)": data["wind"]["deg"],
            "Cloudiness (%)": data["clouds"]["all"],
            "Visibility (m)": data.get("visibility", 0),
            "Report Time": datetime.utcfromtimestamp(data["dt"]).strftime('%Y-%m-%d %H:%M:%S'),
            "Sunrise": datetime.utcfromtimestamp(data["sys"]["sunrise"]).strftime('%Y-%m-%d %H:%M:%S'),
            "Sunset": datetime.utcfromtimestamp(data["sys"]["sunset"]).strftime('%Y-%m-%d %H:%M:%S'),
            "Timezone (s)": data.get("timezone", 0),
        }
    else:
        print(f"Error fetching data for {lat}, {lon}: {response.status_code}")
        return None
    
weather_data = []

for i, row in tqdm(bogota.iterrows(), total=len(bogota)):
    lat, lon = row["LAT"], row["LNG"]
    weather_info = get_weather_data(lat, lon)
    
    if weather_info:
        weather_data.append(weather_info)

    time.sleep(1)  
    
weather_df = pd.DataFrame(weather_data)
weather_df.to_csv("Bogota_weather_data.csv", index=False)

100%|██████████| 59/59 [01:18<00:00,  1.32s/it]


In [7]:
weather_df.head()

,Latitude,Longitude,City,Country,Weather,Description,Temperature (°C),Feels Like (°C),Min Temp (°C),Max Temp (°C),Pressure (hPa),Humidity (%),Wind Speed (m/s),Wind Direction (°),Cloudiness (%),Visibility (m),Report Time,Sunrise,Sunset,Timezone (s)
0,4.642811,-74.144544,Villa Mejía,CO,Clouds,few clouds,13.14,12.49,13.14,13.14,1018,76,3.6,110,20,10000,2025-03-24 01:38:25,2025-03-23 10:59:18,2025-03-23 23:06:44,-18000
1,4.641695,-74.060087,La Merced,CO,Clouds,few clouds,13.04,12.44,13.04,13.04,1019,78,3.6,110,20,10000,2025-03-24 01:38:27,2025-03-23 10:58:58,2025-03-23 23:06:23,-18000
2,4.684074,-74.091357,Santa Barbara Central,CO,Clouds,few clouds,13.14,12.52,13.14,13.14,1019,77,3.6,110,20,10000,2025-03-24 01:38:28,2025-03-23 10:59:05,2025-03-23 23:06:31,-18000
3,4.619381,-74.067988,La Merced,CO,Clouds,few clouds,12.96,12.37,12.96,12.96,1019,79,3.6,110,20,10000,2025-03-24 01:38:29,2025-03-23 10:59:00,2025-03-23 23:06:25,-18000
4,4.603926,-74.177953,Soacha,CO,Clouds,few clouds,13.05,12.42,13.05,13.05,1018,77,3.6,110,20,10000,2025-03-24 01:38:31,2025-03-23 10:59:26,2025-03-23 23:06:51,-18000


In [8]:
import folium

mapa = folium.Map(location=[4.65, -74.1], zoom_start=12)

for _, row in weather_df.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    temperatura = row["Temperature (°C)"]
    descripcion = row["Description"]
    
    popup_text = f"{row['City']}<br>🌡️ {temperatura}°C<br>☁️ {descripcion}"
    
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color="blue", icon="cloud"),
    ).add_to(mapa)

mapa.save("bogota_weather_map.html")
mapa
